I thought in this competition train and test data would be disributed identically, but I decided to test it anyway.
To my surprize, simple adversarial validation (see below) produced AUC of 0.853, and more advanced neural network model achieved AUC of 0.919!
Next question is - how to reflect this information in modeling.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb


# read data
train = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv')
test  = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv')
train['target'] = 1
test['target'] = 0
train = train.append(test).reset_index(drop=True)


# prep data
y  = np.array(train['target']).astype(np.float32)
cols = train.drop(['id','target'], axis=1).columns
x = np.array(train[cols]).astype(np.float32)


# folds
NF = 2
fi = np.zeros(x.shape[1])
params = {'seed': 13, 'objective': 'binary', 'verbose': -1, 'num_leaves': 127}
kf = StratifiedKFold(n_splits=NF, shuffle=True, random_state=13)
oof_pred  = np.zeros(x.shape[0], dtype=np.float32)
for fold, (train_idx, valid_idx) in enumerate(kf.split(x, y)):
    x_train, x_valid = x[train_idx], x[valid_idx]
    y_train, y_valid = y[train_idx], y[valid_idx]
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_valid = lgb.Dataset(x_valid, y_valid)
    model = lgb.train(params, lgb_train, 200, valid_sets=[lgb_train, lgb_valid], early_stopping_rounds=100, verbose_eval=100)
    fi += model.feature_importance(importance_type='gain', iteration=model.best_iteration)
    oof_pred[valid_idx] = model.predict(x_valid).ravel()
    CV = roc_auc_score(y_valid, oof_pred[valid_idx])
    print('fold', fold, 'AUC', np.round(CV,3))
CV = roc_auc_score(y, oof_pred)
print('AUC', np.round(CV,3))
fi = fi / fi.sum()
feature_imp = pd.DataFrame(zip(fi, list(cols)), columns=['Value','Feature'])
feature_imp = feature_imp.sort_values(by='Value').reset_index(drop=True)
feature_imp['Value'] = np.round(feature_imp['Value'], 4)
print(feature_imp)